In [8]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats
import warnings

warnings.filterwarnings('ignore')

In [9]:
df = pd.read_csv('../Health_cleaned_income_delta.csv', low_memory=False)
df.shape

(170761, 63)

In [10]:
df.shape

(170761, 63)

In [11]:
columns_to_int = ['rmstat', 'ragender', 'rahispan', 'raracem', 'ragey_b', 'sagey_b', 'rhltc', 'rhlthlm', 'rhibpe',
        'rdiabe', 'rcancre', 'rlunge', 'rhearte', 'rstroke', 'rpsyche', 'rarthre', 'rhosp', 'rhspnit', 'oop_spend',
        'rlbrf', 'rjphys', 'rjlift', 'rjweeks', 'rjweek2', 'rjcten', 'index_wave', 'insured_gov', 'uninsured',
        'retired', 'collegeplus', 'year', 'inter_year', 'year_of_birth', 'without_work', 'n_jobs', 'broken']

In [12]:
def to_int(el):
    try:
        return int(el)
    except ValueError:
        return el

In [13]:
for col in columns_to_int:
    df[col] = df[col].apply(to_int)

In [14]:
amount_of_answers = df['hhidpn'].value_counts().value_counts().sort_index()
amount_of_answers

1     2837
2     7583
3     1897
4     1919
5     3728
6     1565
7     1481
8     4023
9      845
10     989
11    4665
Name: hhidpn, dtype: int64

In [15]:
# plotly bar plot of amount of answers per person
fig = go.Figure(data=[go.Bar(
    x=amount_of_answers.index,
    y=amount_of_answers.values,
    text=amount_of_answers.values,
    textposition='auto',
)])
fig.update_layout(
    width=900, height=600,
    title='Количество ответов в опросе',
    xaxis_title='Количество ответов',
    yaxis_title='Количество людей',
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    )
)
fig.show()

In [115]:
# cumulative distribution function of amount of answers per person
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=amount_of_answers.index,
        y=amount_of_answers.values.cumsum() / amount_of_answers.values.sum(),
        mode='none',
        fill='tozeroy',
))

fig.update_layout(
    width=900, height=600,
    title='Доля респондентов, которые дали не больше X ответов',
    xaxis_title='Количество ответов',
    yaxis_title='Доля респондентов',
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    )
)

fig.show()

In [117]:
birth_year_by_id = df.groupby('hhidpn')['year_of_birth'].unique()
birth_year_by_id = birth_year_by_id.apply(lambda x: x[0])

In [108]:
# plotly histplot of birth year
fig = go.Figure(data=[go.Histogram(
    x=birth_year_by_id,
)])

fig.update_traces(opacity=0.75)

fig.update_layout(
    width=900, height=600,
    title='Распределение года рождения',
    xaxis_title='Год рождения',
    yaxis_title='Количество людей',
    bargap=0.1
)
fig.show()

In [122]:
birth_year_by_id_gender = df.groupby(['ragender', 'hhidpn'])['year_of_birth'].unique()
birth_year_by_id_gender = birth_year_by_id_gender.apply(lambda x: x[0])

In [130]:
birth_year_by_id_gender.loc[1].value_counts().sort_index()

1897    1
1898    2
1900    2
1901    3
1902    3
       ..
1982    1
1983    1
1984    1
1985    1
1993    1
Name: year_of_birth, Length: 87, dtype: int64

In [135]:
# plotly histplot of birth year
fig = go.Figure()

men_birth_year = birth_year_by_id_gender.loc[1].value_counts().sort_index()
fig.add_trace(go.Scatter(
    x=men_birth_year.index,
    y=men_birth_year.values,
    fill='tozeroy',
    name='Мужчины'
))

women_birth_year = birth_year_by_id_gender.loc[2].value_counts().sort_index()
fig.add_trace(go.Scatter(
    x=women_birth_year.index,
    y=women_birth_year.values,
    fill='tonexty',
    name='Женщины'
))

fig.update_traces(opacity=0.75)

fig.update_layout(
    width=900, height=600,
    title='Распределение года рождения по полам',
    xaxis_title='Год рождения',
    yaxis_title='Количество людей',
    bargap=0.1
)
fig.show()

In [16]:
marriage_groups = ['married', 'separated', 'dead spouse', 'never married', 'divorced']
age_groups = ['young', 'middle', 'old', 'dead outside']

In [25]:
rmstat_df = df[(df['rmstat'] != '.m') & (df['rhltc'] != '.m') & (df['index_wave'] == 2)]

In [26]:
rmstat_df['rmstat'].unique()

array([5, 7, 1, 8, 4, 3, 2, 6], dtype=object)

In [27]:
def calculate_confidence_interval(df, group_name, groups, column, confidence=0.99):
    errors = []

    for group in groups:
        group_df = df[df[group_name] == group]
        n = len(group_df)
        mean, se = group_df[column].mean(), stats.sem(group_df[column])
        h = se * stats.t._ppf((1 + confidence) / 2., n-1)
        errors.append(h)
    
    return errors

In [30]:
fig = go.Figure()

groups = [x for x in range(1, 9)]
colors = ['#6edb85', '#940909', '#79f8fc', '#5d048a', '#273b61', '#043d15', '#303640', '#c1b8f2']

errors = calculate_confidence_interval(rmstat_df, 'rmstat', groups, 'rhltc')
tmp = rmstat_df[rmstat_df['ragey_b'] <= 67]

group_data = tmp.groupby('rmstat')['rhltc'].mean()
all_mean_rhltc = tmp['rhltc'].mean()

fig.add_trace(go.Bar(
    x=group_data.index,
    y=group_data.values,
    error_y=dict(type='data', array=errors, color='#160224', visible=True),
    width=[0.6 for _ in range(8)],
    # texttemplate=[f'{x:.2f}' for x in tmp.groupby('rmstat')['ragey_b'].mean().reindex(groups).values],
    textposition='outside',
    marker_color=colors
))

fig.update_layout(
    title='Среднее изменение здоровья в разных семейных статусах(до 67 лет)',
    xaxis_title='Семейный статус',
    yaxis_title='Среднее значение rhltc',
    width=900, height=600
)

fig.update_yaxes(range=[-0.5, 0.2])

fig.add_hline(
    y=all_mean_rhltc,
    line_dash='dash',
    line_color='black',
    line_width=1,
)

fig.add_annotation(x=7.5, y=-0.14,
    text=f'Среднее по всем группам: {all_mean_rhltc:.2f}',
    showarrow=True,
    arrowhead=1,
    ay=120)

fig.show()

In [31]:
rmstat_df[rmstat_df['rmstat'] == 6]

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,sipena_delta,rgov_delta,sgov_delta,total_work_income_delta,total_pension_income_delta,total_gov_income_delta,total_income_delta,broken,marriage_group,age_group
105568,501809010,6,1,0,2,2006-05-15,55,.m,-1,1,...,NaN,16.671952,NaN,NaN,NaN,16.671952,16.671952,0,separated,middle
160136,529274010,6,2,1,2,2012-07-15,57,.m,1,1,...,NaN,52.567047,NaN,NaN,NaN,52.567047,52.567047,0,separated,middle
165697,534483010,6,2,1,3,2012-09-15,55,.m,1,0,...,NaN,NaN,NaN,50.364483,NaN,NaN,50.364483,0,separated,middle
167363,902203010,6,2,1,3,2012-10-15,53,.m,-1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,separated,middle


~~todo: на график выше добавить средний возраст в каждый столбец, графически выделить плохие и хорошие группы~~

In [62]:
fig = go.Figure()

group_column = 'marriage_group'
colors = ['#6edb85', '#940909', '#303640', '#c1b8f2', '#273b61', '#043d15']
group_data = tmp.groupby(group_column)['rhltc'].mean().reindex(marriage_groups)

fig.add_trace(go.Bar(
    x=group_data.index,
    y=group_data.values,
    width=[0.6 for _ in range(len(marriage_groups))],
    texttemplate=[f'{x:.2f}' for x in tmp.groupby(group_column)['rhltc'].mean().reindex(marriage_groups).values],
    textposition='outside',
    marker_color=colors
))

fig.update_layout(
    title='Среднее изменение здоровья в разных семейных группах(до 67 лет)',
    xaxis_title='Группа',
    yaxis_title='Среднее значение rhltc',
    width=900, height=600
)

fig.update_yaxes(range=[-0.5, 0])

fig.show()

In [ ]:
rmstat_df['ragender'].replace({1: 'male', 2: 'female'}, inplace=True)

In [64]:
rmstat_df.columns

Index(['hhidpn', 'rmstat', 'ragender', 'rahispan', 'raracem', 'riwbegy',
       'ragey_b', 'sagey_b', 'rhltc', 'rhlthlm', 'rhibpe', 'rdiabe', 'rcancre',
       'rlunge', 'rhearte', 'rstroke', 'rpsyche', 'rarthre', 'rhosp',
       'rhspnit', 'oop_spend', 'riearn', 'ripena', 'siearn', 'sipena', 'rcovr',
       'rcovs', 'rlbrf', 'rjphys', 'rjlift', 'rjweeks', 'rjweek2', 'rjcten',
       'index_wave', 'insured_gov', 'uninsured', 'retired', 'collegeplus',
       'year', 'cpi', 'rgov', 'sgov', 'inter_year', 'year_of_birth',
       'total_work_income', 'total_pension_income', 'total_gov_income',
       'total_income', 'without_work', 'n_jobs', 'riearn_delta',
       'siearn_delta', 'ripena_delta', 'sipena_delta', 'rgov_delta',
       'sgov_delta', 'total_work_income_delta', 'total_pension_income_delta',
       'total_gov_income_delta', 'total_income_delta', 'broken',
       'marriage_group', 'age_group'],
      dtype='object')

In [67]:
def hypothesis_check(t, p): 
    if (abs(t) > 2.59) and (p < 0.01): 
        print('Отвергаем Н0. Разница между средними статистически значима. На уровне значимости 99%') 
    elif (abs(t) > 1.9667) and (p < 0.05): 
        print('Отвергаем Н0. Разница между средними статистически значима. На уровне значимости 95%') 
    else: 
        print('Не удалось отвергнуть H0')

In [68]:
df_married = rmstat_df[rmstat_df['marriage_group'] == 'married']
df_separated = rmstat_df[rmstat_df['marriage_group'] == 'separated']
df_dead = rmstat_df[rmstat_df['marriage_group'] == 'dead spouse']
df_never = rmstat_df[rmstat_df['marriage_group'] == 'never married']
df_divorced = rmstat_df[rmstat_df['marriage_group'] == 'divorced']

In [69]:
t, p = stats.ttest_ind(df_married['rhltc'], df_separated['rhltc'], equal_var=False)
var_1 = df_married['rhltc'].var() 
var_2 = df_separated['rhltc'].var() 
n_1 = len(df_married['rhltc']) 
n_2 = len(df_separated['rhltc']) 
s_m1_m2 = np.sqrt(var_1/n_1 + var_2/n_2) 
df_welch = (var_1 + var_2)**2 / (var_1**2 / (n_1 - 1) + var_2**2 / (n_2 - 1)) 
if df_welch > 350: 
    hypothesis_check(t, p) 
else: 
    print('ddof =', df_welch) 
print('ddof =', int(df_welch)) 
print('t =', t) 
print('p-value =', p) 
print('Среднее среди совместно проживающих или женатых =', df_married['rhltc'].mean()) 
print('Среднее среди все сложно =', df_separated['rhltc'].mean())

Отвергаем Н0. Разница между средними статистически значима. На уровне значимости 99%
ddof = 9997
t = 2.740833856309917
p-value = 0.0061581371253321025
Среднее среди совместно проживающих или женатых = -0.1073157401623985
Среднее среди все сложно = -0.14367651276168628


In [70]:
t, p = stats.ttest_ind(df_married['rhltc'], df_dead['rhltc'], equal_var=False)
var_1 = df_married['rhltc'].var() 
var_2 = df_dead['rhltc'].var() 
n_1 = len(df_married['rhltc']) 
n_2 = len(df_dead['rhltc']) 
s_m1_m2 = np.sqrt(var_1/n_1 + var_2/n_2) 
df_welch = (var_1 + var_2)**2 / (var_1**2 / (n_1 - 1) + var_2**2 / (n_2 - 1)) 
if df_welch > 350: 
    hypothesis_check(t, p) 
else: 
    print('ddof =', df_welch) 
print('ddof =', int(df_welch)) 
print('t =', t) 
print('p-value =', p) 
print('Среднее среди совместно проживающих или женатых =', df_married['rhltc'].mean()) 
print('Среднее среди потерявших супруга =', df_dead['rhltc'].mean())

Отвергаем Н0. Разница между средними статистически значима. На уровне значимости 99%
ddof = 85035
t = 33.604936541570936
p-value = 1.2691309889218042e-244
Среднее среди совместно проживающих или женатых = -0.1073157401623985
Среднее среди потерявших супруга = -0.26026261688440877


In [71]:
t, p = stats.ttest_ind(df_married['rhltc'], df_never['rhltc'], equal_var=False)
var_1 = df_married['rhltc'].var() 
var_2 = df_never['rhltc'].var() 
n_1 = len(df_married['rhltc']) 
n_2 = len(df_never['rhltc']) 
s_m1_m2 = np.sqrt(var_1/n_1 + var_2/n_2) 
df_welch = (var_1 + var_2)**2 / (var_1**2 / (n_1 - 1) + var_2**2 / (n_2 - 1)) 
if df_welch > 350: 
    hypothesis_check(t, p) 
else: 
    print('ddof =', df_welch) 
print('ddof =', int(df_welch)) 
print('t =', t) 
print('p-value =', p) 
print('Среднее среди совместно проживающих или женатых =', df_married['rhltc'].mean()) 
print('Среднее среди никогда не женатых =', df_never['rhltc'].mean())

Отвергаем Н0. Разница между средними статистически значима. На уровне значимости 95%
ddof = 17643
t = 2.5701908229720227
p-value = 0.01019058537571036
Среднее среди совместно проживающих или женатых = -0.1073157401623985
Среднее среди никогда не женатых = -0.13258561986781495


In [72]:
t, p = stats.ttest_ind(df_married['rhltc'], df_divorced['rhltc'], equal_var=False)
var_1 = df_married['rhltc'].var() 
var_2 = df_divorced['rhltc'].var() 
n_1 = len(df_married['rhltc']) 
n_2 = len(df_divorced['rhltc']) 
s_m1_m2 = np.sqrt(var_1/n_1 + var_2/n_2) 
df_welch = (var_1 + var_2)**2 / (var_1**2 / (n_1 - 1) + var_2**2 / (n_2 - 1)) 
if df_welch > 350: 
    hypothesis_check(t, p) 
else: 
    print('ddof =', df_welch) 
print('ddof =', int(df_welch)) 
print('t =', t) 
print('p-value =', p) 
print('Среднее среди совместно проживающих или женатых =', df_married['rhltc'].mean()) 
print('Среднее среди разведенных =', df_divorced['rhltc'].mean())

Отвергаем Н0. Разница между средними статистически значима. На уровне значимости 99%
ddof = 41241
t = 6.213608089349647
p-value = 5.300175988549502e-10
Среднее среди совместно проживающих или женатых = -0.1073157401623985
Среднее среди разведенных = -0.14776958458056508


In [106]:
y_ticks = [-1, -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0, 0.1, 0.2]

mean_in_group = rmstat_df['rhltc'].mean()

fig = go.Figure()

tmp = rmstat_df[rmstat_df['marriage_group'] == 'married']
fig.add_trace(go.Bar(name='Married',
    x=age_groups,
    y=tmp.groupby('age_group')['rhltc'].mean().reindex(age_groups),
    # marker_color='rgb(85, 177, 242)'
    texttemplate=[f'{x:.0f}' for x in tmp.groupby('age_group')['ragey_b'].mean().reindex(age_groups).values],
    # texttemplate=['text1', 'text2', 'text3', 'text4', 'text5'],
    textposition='outside',
))

tmp = rmstat_df[rmstat_df['marriage_group'] == 'separated']
fig.add_trace(go.Bar(name='Separated',
    x=age_groups,
    y=tmp.groupby('age_group')['rhltc'].mean().reindex(age_groups), 
    # marker_color='rgb(242, 111, 85)'
    texttemplate=[f'{x:.0f}' for x in tmp.groupby('age_group')['ragey_b'].mean().reindex(age_groups).values],
    textposition='outside',
))

tmp = rmstat_df[rmstat_df['marriage_group'] == 'dead spouse']
fig.add_trace(go.Bar(name='Dead Spouse',
    x=age_groups,
    y=tmp.groupby('age_group')['rhltc'].mean().reindex(age_groups), 
    # marker_color='rgb(242, 111, 85)'
    texttemplate=[f'{x:.0f}' for x in tmp.groupby('age_group')['ragey_b'].mean().reindex(age_groups).values],
    textposition='outside',
))

tmp = rmstat_df[rmstat_df['marriage_group'] == 'never married']
fig.add_trace(go.Bar(name='Never Married',
    x=age_groups,
    y=tmp.groupby('age_group')['rhltc'].mean().reindex(age_groups), 
    # marker_color='rgb(242, 111, 85)'
    texttemplate=[f'{x:.0f}' for x in tmp.groupby('age_group')['ragey_b'].mean().reindex(age_groups).values],
    textposition='outside',
))

tmp = rmstat_df[rmstat_df['marriage_group'] == 'divorced']
fig.add_trace(go.Bar(name='Divorced',
    x=age_groups,
    y=tmp.groupby('age_group')['rhltc'].mean().reindex(age_groups), 
    # marker_color='rgb(242, 111, 85)'
    texttemplate=[f'{x:.0f}' for x in tmp.groupby('age_group')['ragey_b'].mean().reindex(age_groups).values],
    textposition='outside',
))

fig.update_layout(
    barmode='group',
    width=900, height=600,
    title=f'Среднее изменение здоровья по полу и группе возраста в каждой группе брака',
    xaxis_title='Группа возраста',
    yaxis_title='Среднее изменение здоровья',
    yaxis = dict(
        tickmode = 'array',
        tickvals = y_ticks,
        ticktext = y_ticks
    )
)
fig.update_yaxes(range=[-0.5, 0.4])

fig.add_annotation(text="*Над столбцами: средний возраст в группе",
    xref="paper", yref="paper",
    x=1, y=1, showarrow=False
)

fig.show()

In [82]:
rmstat_df[(rmstat_df['age_group'] == 'young') & (rmstat_df['marriage_group'] == 'married')].shape

(949, 63)

In [83]:
rmstat_df[(rmstat_df['age_group'] == 'young') & (rmstat_df['marriage_group'] == 'separated')].shape

(21, 63)

In [84]:
rmstat_df[(rmstat_df['age_group'] == 'young') & (rmstat_df['marriage_group'] == 'dead spouse')].shape

(24, 63)

In [85]:
rmstat_df[(rmstat_df['age_group'] == 'young') & (rmstat_df['marriage_group'] == 'never married')].shape

(9, 63)

In [86]:
rmstat_df[(rmstat_df['age_group'] == 'young') & (rmstat_df['marriage_group'] == 'divorced')].shape

(23, 63)